In [1]:
import numpy as np
import pandas as pd
import wrangle_anomaly as w

import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
traffic = w.wrangle_web_traffic()

opening file from local directory


In [28]:
traffic

,path,user_id,cohort_id,ip,cohort_name,start_date,end_date,program_id,lesson
datetime,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,java-ii
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,java-ii
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,slides
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0,javascript-i
...,...,...,...,...,...,...,...,...,...
2021-04-21 16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0,jquery
2021-04-21 16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0,jquery
2021-04-21 16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0,jquery


# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [32]:
traffic.program_id.value_counts(dropna=False)

2.0    713365
3.0    103411
1.0     30548
4.0         5
Name: program_id, dtype: int64

In [21]:
traffic.reset_index().groupby(['lesson', 'program_id']).\
    datetime.nunique().sort_values(ascending=False).head(50)

lesson                program_id
javascript-i          2             108425
html-css              2              77920
mysql                 2              75352
jquery                2              55871
spring                2              52438
java-iii              2              51626
java-ii               2              50487
java-i                2              37567
                      2              35622
javascript-ii         2              35483
appendix              2              30071
toc                   2              17300
search                2              15141
examples              2              13580
                      3               8228
sql                   3               7447
classification        3               6837
slides                2               6437
3-sql                 3               6099
fundamentals          3               5673
python                3               5559
content               1               5240
1-fundamentals       

In [20]:
traffic[traffic.program_id == 4].reset_index().groupby(['lesson', 'program_id']).\
    datetime.nunique().sort_values(ascending=False).head(50)

lesson   program_id
content  4             3
         4             1
Name: datetime, dtype: int64

In [31]:
traffic[traffic.program_id.isna() == True].reset_index().groupby(['lesson']).\
    datetime.nunique().sort_values(ascending=False).head(50)

Series([], Name: datetime, dtype: int64)

### The most commonly accessed lesson for each program were:
- Program 1 - 'content' which I don't think is a lesson, 'javascript-i' was the next highest
- Program 2 - 'javascript-i'
- Program 3 - 'sql'
- Program 4 - 'content'

# 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [86]:
lesson_df1 = pd.DataFrame(traffic.reset_index().groupby('lesson').\
     cohort_name.value_counts()).rename(columns={'cohort_name':'count'}).\
    reset_index()

In [87]:
lesson_df1.head()

,lesson,cohort_name,count
0,,Staff,6340
1,,Darden,2980
2,,Voyageurs,2101
3,,Fortuna,2038
4,,Bayes,1967


In [90]:
lesson_df2 = pd.DataFrame(traffic.reset_index().groupby('lesson').\
     cohort_name.value_counts(normalize=True)).rename(columns={'cohort_name':'proba'}).\
    reset_index()

In [91]:
lesson_df2.head()

,lesson,cohort_name,proba
0,,Staff,0.138265
1,,Darden,0.064989
2,,Voyageurs,0.045819
3,,Fortuna,0.044445
4,,Bayes,0.042897


In [92]:
# merge the two data frames create above into a single one:
lesson_df = pd.concat([lesson_df1, lesson_df2.proba], axis=1)
lesson_df.head()

,lesson,cohort_name,count,proba
0,,Staff,6340,0.138265
1,,Darden,2980,0.064989
2,,Voyageurs,2101,0.045819
3,,Fortuna,2038,0.044445
4,,Bayes,1967,0.042897


# 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

# 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? 

### 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

In [5]:
traffic.lesson.value_counts()

javascript-i             113838
html-css                  81490
mysql                     79416
jquery                    58317
spring                    55481
                          ...  
stats-assessment              1
statistics-assessment         1
javascript-functions          1
ui                            1
6.03_Explore                  1
Name: lesson, Length: 687, dtype: int64

In [32]:
id_counts = traffic.reset_index().groupby('user_id')[[
    'lesson', 'cohort_name','datetime']].nunique()

In [40]:
cohort_counts = traffic.reset_index().groupby('cohort_name')[[
    'lesson', 'user_id','datetime']].nunique()

In [34]:
cohort_counts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Andromeda to Zion
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   lesson       47 non-null     int64
 1   cohort_name  47 non-null     int64
 2   datetime     47 non-null     int64
dtypes: int64(3)
memory usage: 1.5+ KB


In [41]:
cohort_counts

,lesson,user_id,datetime
name,,,
Andromeda,41,28,24948
Apex,28,24,32969
Apollo,2,1,4
Arches,35,18,8529
Badlands,8,3,90
Bash,39,23,17487
Bayes,86,23,24301
Betelgeuse,36,22,28833
Ceres,40,29,39647


In [46]:
last_access = traffic.reset_index().groupby('user_id').datetime.max()

user_id
1     2021-04-21 10:32:48
2     2018-05-10 18:25:38
3     2019-04-06 21:32:32
4     2018-03-27 14:28:18
5     2018-07-19 14:08:20
              ...        
977   2021-04-21 09:46:37
978   2021-04-21 10:21:40
979   2021-04-21 10:53:46
980   2021-04-19 12:08:50
981   2021-04-21 11:01:23
Name: datetime, Length: 911, dtype: datetime64[ns]

# 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

# 7. Which lessons are least accessed?